https://arxiv.org/abs/2004.11362

In [ ]:
import numpy as np
import torch

In [ ]:
class SCL_out(torch.nn.Module):
    def __init__(self, temp=1.):
        super().__init__()
        self.temp = temp
    
    def forward(self, y_pred, y_true):
        ran = np.arange(y_pred.shape[0])
        loss = 0.
        for i, anchor in enumerate(y_pred):
            class_loss = 0.
            
            prob = (y_true == y_true[i]).sum()
            for j, other in enumerate(y_pred):
                if i != j and y_true[i] == y_true[j]:
                    A_i = y_pred[~np.isin(ran, [i, j])]
                    denum = torch.log(torch.exp(A_i @ anchor / self.temp).sum())
                    class_loss += (denum - other.T @ anchor / self.temp)
            class_loss /= prob
            loss += class_loss
        return loss

In [ ]:
class SCL_in(torch.nn.Module):
    def __init__(self, temp=1.):
        super().__init__()
        self.temp = temp
    
    def forward(self, y_pred, y_true):
        ran = np.arange(y_pred.shape[0])
        loss = 0.
        for i, anchor in enumerate(y_pred):
            class_loss = 0.
            prob = (y_true == y_true[i]).sum()
            for j, pos in enumerate(y_pred):
                if i != j and y_true[i] == y_true[j]:
                    A_i = y_pred[~np.isin(ran, [i, j])]
                    denum = torch.exp(A_i @ anchor / self.temp).sum()
                    num = torch.exp(torch.dot(pos, anchor) / self.temp)
                    class_loss += num / denum
            loss += torch.log(prob / class_loss)
        return loss

In [ ]:
class SSCL(torch.nn.Module):
    def __init__(self, temp=1.):
        super().__init__()
        self.temp = temp
    
    def forward(self, y_pred, y_true, mapper):
        ran = np.arange(y_pred.shape[0])
        loss = 0.
        for i, anchor in enumerate(y_pred):
            pos = y_pred[mapper[i]]
            A_i = y_pred[~np.isin(ran, [i, mapper[i]])]
            denum = torch.log(torch.exp(A_i @ anchor / self.temp).sum())
            loss += (denum - torch.dot(pos, anchor) / self.temp)
        return loss